# Bayesian Transformation Tutorial

One line → any PyTorch model becomes Bayesian.

In [ ]:
import torch
import torch.nn as nn
from probly.transformation.bayesian import bayesian

## 1. Normal PyTorch model
This is a regular neural network

- Weights are **fixed numbers**  
- Every time you run it on the same input → you get **exactly the same output**  
- It can never say “I’m not sure” — even when it should be!

```python
model(x) → always 0.42

In [ ]:
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 10),
            nn.ReLU(),
            nn.Linear(10, 8),
            nn.Linear(8, 1)
        )
    def forward(self, x):
        return self.net(x)

model = MyModel()
print("Original model:")
print(model)

## 2. One line → Bayesian Test
We call `bayesian(model)`

In one line, Probly automatically finds **every** `nn.Linear` layer and replaces it with a `BayesLinear` layer that has learnable weight distributions.

Now the model has uncertainty built-in — but by default it still returns the **exact same numbers** as before (the mean), so nothing breaks.

In [ ]:
bayesian_model = bayesian(model)
print("\nAfter bayesian() → all Linear → BayesLinear!")
print(bayesian_model)

## 3. By default: deterministic (mean only)
Even though the weights are now distributions, `bayesian(model)` is **smart**.

By default it returns only the **mean** of each distribution → you get **exactly the same output** as the original model (no surprises, no randomness).

This means you can drop it into any existing code and everything keeps working perfectly — while secretly being Bayesian under the hood.

In [ ]:
x = torch.randn(5, 2)

out1 = bayesian_model(x)
out2 = bayesian_model(x)

print("Same input → same output?", torch.allclose(out1, out2))  # True!
print("Output shape:", out1.shape)

## 4. To get uncertainty, run forward pass multiple times (sampling happens inside BayesLinear)
# Each BayesLinear samples internally by default when called

In [ ]:
x = torch.randn(5, 2)

# Run 100 forward passes → each time weights sample differently
samples = torch.stack([bayesian_model(x) for _ in range(100)])

print("Predictive mean:\n", samples.mean(0))
print("Predictive std:\n", samples.std(0))
print("\nUncertainty shown! (std > 0 means sampling is working)")

## 5. Your test passes 

In [ ]:
import os
os.chdir("D:/my_project/ProblyPros")
!pytest tests/probly/transformation/bayesian/test_torch.py -v